In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # setting ignore as a parameter and further adding category
from whispr import *
import os


In [20]:
# gene expression. 2.5ul rxns, 75% txtl
rxn_vol = 2.5
source_plate_type = '384PP_AQ_BP' 

os.getcwd()
folder = os.getcwd() + '/Experiments/240812_KO/'
sp_plasmids_file = folder + '240812_source_plate.xlsx'
sp_plasmids = pd.read_excel(sp_plasmids_file, index_col = 0, engine='openpyxl')
sp_plasmids = sp_plasmids[~sp_plasmids['Well'].isna()]

layout_genex_file = folder + '240812_plate_map.csv'
#layout_genex = pd.xlsx(layout_genex_file, index_col = 0, dtype = str)
layout_genex = pd.read_csv(layout_genex_file, index_col = 0, dtype = str)

mt_genex_file = folder + '240812_table_amounts.csv'
#mt_genex = pd.xlsx(mt_genex_file, index_col = 0, dtype = str).fillna(0)
mt_genex = pd.read_csv(mt_genex_file, index_col = 0, dtype = str).fillna(0)

checkInputs(sp_plasmids, mt_genex,source_plate_type)

vol_table_df = generateVolumeTable(mt_genex, sp_plasmids, rxn_vol = 0.25*rxn_vol, total_vol = rxn_vol, fill_with='Water', multiRpW = False)

#specify rxn_vol (default = 2.5) and total_vol (default = 10) if you'd like to change the volume of each individual replicate or the total reaction volume
protocol_genex_df = writeProtocol(source_plate_type, vol_table_df, layout_genex,sp_plasmids, update_source_vol=folder+'plasmids_sp_updated.xlsx')


Volumes used from each well for this protocol:
{'1_A1': 4.725, '1_A2': 2.1, '1_A3': 5.25, '1_A4': 0.15, '1_A5': 0.075, '1_A6': 0.075, '1_A7': 0.075, '1_A8': 0.075, '1_A9': 0.075, '1_A10': 0.375, '1_A11': 0.15}


In [21]:
protocol_genex_df = protocol_genex_df[0]

In [17]:
vol_table_df[vol_table_df['Label'] == label]
label

'pCK303.AAV_1'

In [18]:
vol_table_df

,Label,Water,pDA010.188,pRC029,pRC008.206,pRC008.208,pRC008.210,pRC008.212,pCK303.306,pCK303.AAV,pRC009,pCK507
0,pRC008_206_1,0.225,0.1,0.25,0.025,0,0,0,0,0,0.025,0
1,pRC008_206_2,0.225,0.1,0.25,0.025,0,0,0,0,0,0.025,0
2,pRC008_206_3,0.225,0.1,0.25,0.025,0,0,0,0,0,0.025,0
3,pRC008_208_1,0.225,0.1,0.25,0,0.025,0,0,0,0,0.025,0
4,pRC008_208_2,0.225,0.1,0.25,0,0.025,0,0,0,0,0.025,0
5,pRC008_208_3,0.225,0.1,0.25,0,0.025,0,0,0,0,0.025,0
6,pRC008_210_1,0.225,0.1,0.25,0,0,0.025,0,0,0,0.025,0
7,pRC008_210_2,0.225,0.1,0.25,0,0,0.025,0,0,0,0.025,0
8,pRC008_210_3,0.225,0.1,0.25,0,0,0.025,0,0,0,0.025,0
9,pRC008_212_1,0.225,0.1,0.25,0,0,0,0.025,0,0,0.025,0


In [22]:
# everything gets txtl

# get destination wells
for layout_id, layout in enumerate([layout_genex]):    
    # reads plate layout and assigns wells to each reaction (rxn_loc; dict)
    labels = pd.unique(np.concatenate(layout.values))
    rxn_loc = {}
    for l in labels:
        if type(l) is str:
            index = layout[layout.isin([l])].stack().index
            rxn_loc[l] = []
            for i in index:
                rxn_loc[l].append(str(i[0]) + str(i[1]))


txtl_well = 'P24'
for label, dwell in rxn_loc.items():
    for d in dwell:
        print(label)
        txtl_vol = vol_table_df[vol_table_df['Label'] == label].iloc[0,1:].sum()/0.25*0.75*1000 # add txtl to make 75%
        row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': txtl_well,
                                    'Destination Plate Name':'Destination[1]', 'Destination Well': d, 'Transfer Volume': txtl_vol}
        protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)

protocol_genex_df.to_csv(folder+'protocol_genex.csv',index = False)
print('TXTL needed: '+str(protocol_genex_df[protocol_genex_df['Source Well'] == txtl_well]['Transfer Volume'].sum()/1000))
# carry out gene expression reaction

pRC008_206_1
pRC008_206_1
pRC008_206_2
pRC008_206_2
pRC008_206_3
pRC008_206_3
pRC008_208_1
pRC008_208_2
pRC008_208_3
pRC008_210_1
pRC008_210_2
pRC008_210_3
pRC008_212_1
pRC008_212_2
pRC008_212_3
pCK303_AAV_1
pCK303_AAV_2
pCK303_AAV_3
pCK303_306_1
pCK303_306_2
pCK303_306_3
TXTL needed: 39.375


In [24]:
# dilution. everything gets Tris, for 20x -> 57uL

##### doing it by hand if not too many reactions #####

# protocol_dilution_df = pd.DataFrame(columns=protocol_genex_df.columns)
# source_plate_type = '6RES_AQ_B' 
# tris_well = 'A1'
# tris_vol = 57*1000

# for dwell in protocol_genex_df['Destination Well'].unique():
#     row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': tris_well,
#                                 'Destination Plate Name':'Destination[1]', 'Destination Well': dwell, 'Transfer Volume': tris_vol}
#     protocol_dilution_df = protocol_dilution_df.append(row, ignore_index = True)

# protocol_dilution_df.to_csv('protocols/221024_arpae_dilutions.csv',index = False)

In [25]:
# from source plate with buffer, mix into 96

# from source plate with genex, mix the diluted txtl

In [39]:
# biosynthesis. 25ul rxns, 2.5ul of diluted txtl

# buffer source plate
sp_buffers_file = folder + 'buffers_sp.xlsx'
sp_buffers= pd.read_excel(sp_buffers_file, index_col = 0, engine = 'openpyxl')
sp_buffers = sp_buffers[~sp_buffers['Well'].isna()]

# HEPES reservoir (actually same plate as Tris)
sp_hepes = pd.DataFrame(columns=sp_buffers.columns)
sp_hepes = sp_hepes.append({'Label':'HEPES', 'Well':'A2,A3', 'Concentration':'', 'Volume':'2000,2000'}, ignore_index = True)

# gene expression source plate (diluted w/ Tris)
sp_genex = sp_from_layout(layout_genex, 60)

# group source plates
sp_types = ['384PP_AQ_BP','6RES_AQ_BP2','384PP_AQ_BP'] # triple check this
sps = [sp_buffers,sp_hepes,sp_genex]

# get mixing table
mt_biosyn_file = folder + '040323_mixing-table-biosyn.csv'
mt_biosyn = pd.read_csv(mt_biosyn_file, index_col = 0, dtype = str).fillna(0)
mt_biosyn = mt_biosyn[~mt_biosyn.index.isna()]

# check formats
checkInputs(sps,mt_biosyn,sp_types)

# layouts for destination plate(s)
layout_biosyn1_file = folder + '040323_plate-layout-biosyn.csv' 
layout_biosyn1 = pd.read_csv(layout_biosyn1_file, index_col = 0, dtype = str)
layout_biosyn1 = layout_biosyn1[~layout_biosyn1.index.isna()]

# filename = '221018_arpae_biosyn_neg.csv'
# layout_biosyn2_file = 'plate_layouts/'+filename 
# layout_biosyn2 = pd.read_csv(layout_biosyn2_file, index_col = 0, dtype = str)
# layouts = [layout_biosyn1,layout_biosyn2]

vol_table_df = generateVolumeTable(mt_biosyn, sps, rxn_vol = 22.5, total_vol = 25, fill_with='HEPES')

protocol_biosyn_dfs = writeProtocol(sp_types, vol_table_df, layout_biosyn1,sps, update_source_vol= folder +'combined_sp_updated.xlsx')

#protocol_biosyn_df.to_csv('protocols/221018_arpae_biosyn.csv',index = False)

NameError: Need more volume of tdcb_2.5 to complete reaction ser_0. Add another well to source plate.

In [27]:
protocol_biosyn_dfs[0].to_csv(folder+'biosyn_protocol_384.csv',index = False)
protocol_biosyn_dfs[1].to_csv(folder+'biosyn_protocol_6RES.csv',index = False)